In [1]:
!nvidia-smi

Sun Jan 25 18:12:16 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

# Get RT-DETR
if not os.path.isdir("/content/RT-DETR"):
  %cd /content
  print("Cloning RT-DETR")
  !git clone https://github.com/lyuwenyu/RT-DETR.git
else:
  print("RT-DETR is already cloned.")

%cd RT-DETR/rtdetrv2_pytorch
!pip install --upgrade pip
!pip install -r requirements.txt
%cd ..

# Get pretrained weights <- starting training from here
if not os.path.isfile("/content/RT-DETR/pretrained/rtdetrv2_r18vd_120e_coco.pth"):
  !mkdir -p pretrained
  !wget -P pretrained https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth

/content
Cloning RT-DETR
Cloning into 'RT-DETR'...
remote: Enumerating objects: 1119, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 1119 (delta 13), reused 5 (delta 5), pack-reused 1084 (from 3)
Receiving objects: 100% (1119/1119), 669.11 KiB | 2.91 MiB/s, done.
Resolving deltas: 100% (527/527), done.
/content/RT-DETR/rtdetrv2_pytorch
/content/RT-DETR
--2026-01-25 18:12:26--  https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/676791659/55d7790a-dd84-4ad7-a2e1-461a82160f99?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-25T19%3A05%3A38Z&rscd=attachment%3B+filename%3Drtdetrv2_r18vd_120e_coco.pth&rsct=application%2Foctet-stream&skoid=9

In [9]:
# Copy and unzip data from GDrive
%cd /content
if not os.path.isdir("/content/soccertrack"):
  if not os.path.isfile("/content/soccertrack.zip"):
    print('Copying soccertrack dataset ...')
    !cp /content/drive/MyDrive/datasets/soccertrack.zip .
  print('Unzipping soccertrack dataset ...')
  !unzip -q /content/soccertrack.zip -d /content/soccertrack

  !rm /content/soccertrack.zip

Unzipping soccertrack dataset ...


In [10]:
# Dataset processing
!pip install pandas pillow tqdm opencv-python

In [5]:
import torch
device = torch.cuda.is_available()
print(f"GPU device for training / inference: {device}")

GPU device for training / inference: True
